In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/online_shop.db')

In [ ]:
trans = pd.read_sql('SELECT * FROM transactions', conn)
trans.head()

In [ ]:
items = pd.read_sql('SELECT * FROM items', conn)
items.head()

In [ ]:
# merge de las dos tablas
df = trans.merge(items, how='left', on='item_id')
df

## Preparación para el Modelado

Nuestro objetivo es predecir cuales clientes son más probable que compren comida para perro en Junio. 

Recordemos que cada renglón debe tener la información de un cliente

En términos de machine learning, con aprendizaje supervisado tendríamos:

**y**: Compras de junio

**x**: información anterior, en este caso Abril y Mayo

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
# Convertir purchase_date a datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df.dtypes

In [ ]:
# Quitar '$' a columna price
df['price'] = df['price'].str.replace('$', '', regex=False)
df['price']

In [ ]:
# Convertir a valor numérico la columna price
df['price'] = pd.to_numeric(df['price'])
df['price']

In [ ]:
# Considerar solo la ventas antes de junio
df_abril_mayo = df[df.purchase_date.dt.month < 6]
df_abril_mayo.head()

In [ ]:
# Ventas de junio solamente
df_junio = df[df.purchase_date.dt.month == 6]
df_junio.head()

In [ ]:
df_junio[df_junio['item_description'] == 'Dog Food']

In [ ]:
# Renglones de df_junio donde item_description = 'Dog Food'
df_junio_dog_food = df_junio[df_junio['item_description'] == 'Dog Food']
df_junio_dog_food

In [ ]:
df_junio_dog_food.groupby('customer')['item_id'].count()

In [ ]:
compras_junio = (df_junio_dog_food
                           .groupby('customer')['item_id']
                           .count()
                           .rename('compras_comida_junio')).reset_index()

compras_junio

In [ ]:
# ¿Cuanto ha gastado cada cliente en abril y mayo?
df_abril_mayo.groupby('customer')['price'].sum()

In [ ]:
# ¿Cuanto ha gastado cada cliente en abril y mayo?
total_gastado = df_abril_mayo.groupby('customer')['price'].sum().rename('total_gastado').reset_index()
total_gastado

In [ ]:
# Hacemos Merge
model_df = total_gastado.merge(compras_junio, how = 'outer')
model_df

In [ ]:
df_abril_mayo.head()

## Dummy variables

In [ ]:
df_abril_mayo.head()

## Fechas a valores numéricos

In [ ]:
model_df.head()